## Урок 9. Python применение продвинутых методов.

In [51]:
import numpy as np
import pandas as pd

import scipy.stats as stats
from statsmodels.stats import proportion
import statsmodels.stats.power as smp

import warnings
warnings.filterwarnings('ignore')

### Задание 1

Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.
Mobile Games AB Testing with Cookie Cats

In [4]:
df = pd.read_excel('результаты А_B.xlsx')
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
...,...,...,...,...,...
90184,9999441,gate_40,97,1,0
90185,9999479,gate_40,30,0,0
90186,9999710,gate_30,28,1,0
90187,9999768,gate_40,51,1,0


* **userid** - уникальный номер, который идентифицирует каждого игрока.

* **version** - Был ли игрок поставлен в контрольную группу (gate_30 - ворота 30 уровня) или в группу с перемещенными воротами (gate_40 - ворота 40 уровня).

* **sum_gamerounds** - количество раундов, сыгранных каждым игроком в течение первых 14 дней после установки.

* **retention_1** - Вернулся ли игрок в игру через 1 день после установки.

* **retention_7** - Вернулся ли игрок в игру через 7 дней после установки.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [7]:
df.isna().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

In [8]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


In [9]:
df.groupby('userid', as_index=False).agg({'version': pd.Series.nunique}).query('version > 1')

,userid,version


In [10]:
df.duplicated().sum()

0

In [11]:
df["userid"].nunique()

90189

In [12]:
df[['sum_gamerounds', 'retention_1', 'retention_7']].describe()

,sum_gamerounds,retention_1,retention_7
count,90189.000000,90189.000000,90189.000000
mean,51.872457,0.445210,0.186065
std,195.050858,0.496992,0.389161
min,0.000000,0.000000,0.000000
25%,5.000000,0.000000,0.000000
50%,16.000000,0.000000,0.000000
75%,51.000000,1.000000,0.000000
max,49854.000000,1.000000,1.000000


In [13]:
#Уберем  выброс в поле sum_gamerounds.
df = df[df['sum_gamerounds'] < 49000]

In [15]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018800e+04,90188.000000,90188.000000,90188.000000
mean,4.998397e+06,51.320253,0.445214,0.186056
std,2.883298e+06,102.682719,0.496992,0.389154
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512211e+06,5.000000,0.000000,0.000000
50%,4.995804e+06,16.000000,0.000000,0.000000
75%,7.496461e+06,51.000000,1.000000,0.000000
max,9.999861e+06,2961.000000,1.000000,1.000000


In [14]:
df['version'].value_counts()

gate_40    45489
gate_30    44699
Name: version, dtype: int64

In [16]:
df['retention_1'].value_counts()

0    50035
1    40153
Name: retention_1, dtype: int64

In [17]:
df['retention_7'].value_counts()

0    73408
1    16780
Name: retention_7, dtype: int64

In [18]:
df.groupby('version').agg({'userid': 'count', 'retention_1': 'mean', 'retention_7': 'mean', 'sum_gamerounds': sum})

,userid,retention_1,retention_7,sum_gamerounds
version,,,,
gate_30,44699,0.448198,0.190183,2294941
gate_40,45489,0.442283,0.182000,2333530


* Пустых данных в выборке нет.
* Количество пользователей в группах gate_30/gate_40 примерно одинаковое 
* Дублей в userid нет
* Отрицательных (ошибочных) значений в выборке нет.
* Поле version имеет два значения gate_30/gate_40 - ошибочных значений нет.
* Поля retention_1 и retention_7 содержит два значения: 0/1 - ошибочных значений нет.

In [59]:
mannwhitneyu(df[df.version == 'gate_30'].sum_gamerounds,
             df[df.version == 'gate_40'].sum_gamerounds)

MannwhitneyuResult(statistic=1024285761.5, pvalue=0.05089155279145376)

Так как p-value > 0.05 то мы приманием нулевую гипотезу о равенстве средних значений количества раундов в двух выборках. Т.е. между gate_30 и gate_40 статистически значимых различий нет.

In [55]:
A = df.query('version == "gate_30"')
B = df.query('version == "gate_40"')
n1, n2 = A.shape[0], B.shape[0]


In [57]:
#возвращение игрока в игру через 1 день после установки
m1 = A['retention_1'].sum()
m2 = B['retention_1'].sum()

proportion.proportions_ztest(np.array([m1, m2]), np.array([n1, n2]), alternative='two-sided')

(1.787103509763628, 0.0739207603418346)

Так как p-value > 0.05 то мы приманием нулевую гипотезу о равенстве доли вернувшихся в игру через 1 день в двух выборках.
В группах gate_30 и gate_40 не обнаружены статистически значимые отличия по retention_1.

In [58]:
# возвращение игрока в игру через 7 дней после установки
m1 = A.query('retention_7 == 1')['retention_7'].sum()
m2 = B.query('retention_7 == 1')['retention_7'].sum()

proportion.proportions_ztest(np.array([m1, m2]), np.array([n1, n2]))

(3.1574100858819936, 0.0015917731773993442)

Так как p-value < 0.05 то мы отвергаем нулевую гипотезу о равенстве доли вернувшихся в игру через 7 день в двух выборках.
В группах gate_30 и gate_40 обнаружены статистически значимые отличия по retention_7.

Вывод таков: Не вводить изменения, на которых происходило тестирование группы B.